Esse script é o que adiciona os layers no mapa em loop, dando zoom nos quadrados que também são gerados pelo script.
Não é perfeito, as saidas são o eXport dos layouts, que podem não serem corretamente georeferenciados

In [ ]:
import os
import requests
import time
import arcpy

output_folder = r"C:\Users\davils\Documents\ArcGIS\Projects\API\tempp"
map_name = "Map"
cntd = 0

# Obtenha um projeto do ArcGIS Pro
aprx = arcpy.mp.ArcGISProject("CURRENT")

# Obtenha um mapa dentro do projeto
m = aprx.listMaps()[0]

# if your Planet API Key is not set as an environment variable, you can paste it below
if os.environ.get('PL_API_KEY', ''):
    API_KEY = os.environ.get('PL_API_KEY', '')
else:
    API_KEY = 'PLAKa0657b23d6d04653b63dcb1eefdaaacc'

assert API_KEY and not API_KEY.startswith('PASTE')

In [ ]:
import time

# Projeta os pontos de WGS84 (EPSG:4326) para Web Mercator (EPSG:3857)
arcpy.Project_management(
    in_dataset="pontoss",
    out_dataset="pontos_projetados",
    out_coor_system=arcpy.SpatialReference(3857)  # Web Mercator
)
time.sleep(5)

arcpy.analysis.Buffer(
    in_features="pontos_projetados",
    out_feature_class="buffers_projetados",
    buffer_distance_or_field="1000 Meters",  # Distância precisa em metros
    line_side="FULL",
    line_end_type="ROUND",
    method="GEODESIC"  # Usa cálculo geodésico para CRS projetado
)
time.sleep(5)

arcpy.management.MinimumBoundingGeometry(
    in_features="buffers_projetados",
    out_feature_class="retangulos_projetados",
    geometry_type="RECTANGLE_BY_AREA",
    group_option="NONE"
)
time.sleep(5)

arcpy.management.FeatureVerticesToPoints(
    in_features="retangulos_projetados",
    out_feature_class="vertices_projetados",
    point_location="ALL"
)
time.sleep(5)

# Converte os vértices de volta para WGS84 se precisar de lat/long
arcpy.Project_management(
    in_dataset="vertices_projetados",
    out_dataset="vertices_wgs84",
    out_coor_system=arcpy.SpatialReference(4326)  # WGS84
)
time.sleep(5)

arcpy.management.CalculateGeometryAttributes(
    in_features="vertices_wgs84",
    geometry_property="lat_long POINT_COORD_NOTATION",
    coordinate_format="DD"  # Graus Decimais
)

In [ ]:
# Stockton, CA bounding box (created via geojson.io)
geojson_geometry = {
  "type": "Polygon",
  "coordinates": 
     [
          [
            [
              -43.29520982852239,
              -22.94525882116791
            ],
            [
              -43.29520982852239,
              -22.991522610411707
            ],
            [
              -43.24091986050328,
              -22.991522610411707
            ],
            [
              -43.24091986050328,
              -22.94525882116791
            ],
            [
              -43.29520982852239,
              -22.94525882116791
            ]
          ]
        ]
}

In [ ]:
#import geopandas as gpd

# Ler GeoJSON
#gdf = r"C:\Users\davils\Documents\ArcGIS\Projects\API\rio_de_janeiro_FeaturesToJSO.geojson"

input_dict = {
    'type': 'GeometryFilter',
    'field_name': 'geometry',
    'config': {
        0: '<POLYGON ((-43.767 -22.913, -43.768 -22.931, -43.748 -22.932, -43.748 -22.913, -43.767 -22.913))>'
    }
}

# Extrair a string do polígono do input_dict
polygon_wkt = input_dict['config'][0]

# Limpar a string (remover caracteres indesejados)
cleaned_wkt = polygon_wkt.strip("<>").replace("POLYGON ((", "").replace("))", "")

# Dividir os pontos e converter em coordenadas [[lon, lat], ...]
coordinates = [
    [float(coord.split()[0]), float(coord.split()[1])] 
    for coord in cleaned_wkt.split(", ")
]

# Fechar o polígono (último ponto igual ao primeiro)
if coordinates[0] != coordinates[-1]:
    coordinates.append(coordinates[0])

# Estruturar no formato GeoJSON
geojson_geometry = {
    "type": "Polygon",
    "coordinates": [coordinates]  # Lista de anéis (no caso, 1 anel)
}

print(geojson_geometry)

# get images that overlap with our AOI 
geometry_filter = {
  "type": "GeometryFilter",
  "field_name": "geometry",
  "config": geojson_geometry
}

In [ ]:
# get images that overlap with our AOI 
#     geometry_filter = {
#       "type": "GeometryFilter",
#       "field_name": "geometry",
#       "config": geojson_geometry
#     }

# get images acquired within a date range
date_range_filter = {
  "type": "DateRangeFilter",
  "field_name": "acquired",
  "config": {
    "gte": "2024-08-28T00:00:00.000Z",
    "lte": "2024-09-01T00:00:00.000Z"
  }
}

# only get images which have <50% cloud coverage
cloud_cover_filter = {
  "type": "RangeFilter",
  "field_name": "cloud_cover",
  "config": {
    "lte": 0.5
  }
}

# combine our geo, date, cloud filters
combined_filter = {
  "type": "AndFilter",
  "config": [geometry_filter, date_range_filter, cloud_cover_filter]
}

In [ ]:
import json
import pprint
import requests
from requests.auth import HTTPBasicAuth

item_type = "PSScene"

# API request object
search_request = {
  "item_types": [item_type], 
  "filter": combined_filter
}

# fire off the POST request
search_result = \
  requests.post(
    'https://api.planet.com/data/v1/quick-search',
    auth=HTTPBasicAuth(API_KEY, ''),
    json=search_request)

geojson = search_result.json()

# let's look at the first result
#pprint.pprint(geojson)
#pprint.pprint(geojson['features'][0])

In [ ]:
# extract image IDs only
image_ids = [feature['id'] for feature in geojson['features']]

# For demo purposes, just grab the first image ID
id0 = image_ids[0]
id0_url = 'https://api.planet.com/data/v1/item-types/{}/items/{}/assets'.format(item_type, id0)

# Returns JSON metadata for assets in this ID. Learn more: planet.com/docs/reference/data-api/items-assets/#asset
result = \
  requests.get(
    id0_url,
    auth=HTTPBasicAuth(API_KEY, '')
  )

# List of asset types available for this particular satellite image
print(result.json().keys())

In [ ]:
import os, time, arcpy

# Diretório de saída
output_folder = r"C:\Users\davils\Documents\ArcGIS\Projects\API\tempp"

# Obter o projeto, mapa ativo e seu CRS
aprx = arcpy.mp.ArcGISProject("CURRENT")
mapa = aprx.activeMap
map_crs = mapa.spatialReference  # CRS do mapa
sr = arcpy.SpatialReference(map_crs.factoryCode)
arcpy.env.outputCoordinateSystem = sr

# Seleciona o layout e o map frame que serão usados na exportação
lyt = aprx.listLayouts("Layout10")[0]
map_frame = lyt.listElements("MAPFRAME_ELEMENT")[0]

# Obter a camada dos quadrados ("tabela1_XYTabl_MinimumBoundi")
bound_layer = None
for lyr in mapa.listLayers():
    if lyr.name.lower() == "retangulos_projetados".lower():
        bound_layer = lyr
        break

if bound_layer is None:
    arcpy.AddError("Camada de quadrados 'tabela1_XYTabl_MinimumBoundi' não foi encontrada.")
    raise Exception("Camada de quadrados não encontrada.")

# Obtenha os limites (extents) dos quadrados juntamente com seu CRS usando um SearchCursor.
# Cada item será uma tupla: (extent, spatial_reference)
boundaries = []
with arcpy.da.SearchCursor(bound_layer, ["SHAPE@"]) as cursor:
    for row in cursor:
        geom = row[0]
        boundaries.append( (geom.extent, geom.spatialReference) )

# Template da URL do tile da Planet.
PLANET_TILE_URL_TEMPLATE = "https://tiles.planet.com/data/v1/{item_type}/{item_id}/{{z}}/{{x}}/{{y}}.png?api_key={API_KEY}"

# Contador para as exportações
cntd = 0

# Loop externo: para cada imagem (obtida a partir do seu geojson)
for result in geojson["features"]:
    try:
        item_id = result["id"]
        item_type = result["properties"]["item_type"]
        
        # Cria a URL do tile para a imagem
        tile_url = PLANET_TILE_URL_TEMPLATE.format(item_id=item_id, item_type=item_type, API_KEY="PLAK3d8a218e21f24f9488f40caf51904383")
        print(f"Processando imagem: {item_id}")
        print(f"Tile URL: {tile_url}")
        
        # Adiciona o layer do tile; ele será adicionado como um layer Tiled Internet Layer.
        # Como ele já está funcionando na sua sessão, use-o diretamente.
        added_layer = mapa.addDataFromPath(tile_url)
        time.sleep(2)  # Aguarda o carregamento do layer
        
        # Loop interno: para cada quadrado (bound) da camada de limites
        for idx, (bound_ext, bound_sr) in enumerate(boundaries):
            # Verifica se o CRS do quadrado é diferente do do mapa
            if bound_sr.factoryCode != map_crs.factoryCode:
                # Cria um polígono a partir do extent com o CRS original do bound
                extent_poly = arcpy.Polygon(
                    arcpy.Array([
                        arcpy.Point(bound_ext.XMin, bound_ext.YMin),
                        arcpy.Point(bound_ext.XMin, bound_ext.YMax),
                        arcpy.Point(bound_ext.XMax, bound_ext.YMax),
                        arcpy.Point(bound_ext.XMax, bound_ext.YMin),
                        arcpy.Point(bound_ext.XMin, bound_ext.YMin)
                    ]),
                    bound_sr
                )
                extent_poly_proj = extent_poly.projectAs(sr)
                # Atualiza o extent com o resultado da reprojeção
                bound_ext = extent_poly_proj.extent
                print(f"Extent reprojetado do quadrado {idx}: {bound_ext.XMin}, {bound_ext.YMin}, {bound_ext.XMax}, {bound_ext.YMax}")
            else:
                print(f"Usando extent do quadrado {idx}: {bound_ext.XMin}, {bound_ext.YMin}, {bound_ext.XMax}, {bound_ext.YMax}")
            
            # Atualiza o extent do map frame do layout
            map_frame.camera.setExtent(bound_ext)
            time.sleep(1)  # Aguarda a atualização do layout
            
            # Define o nome do arquivo de saída (pode ser TIFF ou JPEG)
            output_file = os.path.join(output_folder, f"tile_{item_id}_quad{idx}_{cntd}.tif")
            # Exporta o layout para TIFF, utilizando os parâmetros desejados
            lyt.exportToTIFF(
                output_file, 
                resolution=52,  
                world_file=True, 
                geoTIFF_tags=True,
                transparent_background=True
            )
            print(f"Exportado: {output_file}")
            
            # Define a projeção no arquivo exportado (caso os metadados não estejam embedados)
            arcpy.DefineProjection_management(output_file, sr)
            print(f"Projeção definida para: {output_file}")
            
            cntd += 1
        
        # Remove a camada temporária da imagem após processar todos os quadrados
        mapa.removeLayer(added_layer)
        time.sleep(1)
        
    except Exception as e:
        print(f"Erro no item {item_id}: {e}")
        try:
            mapa.removeLayer(added_layer)
        except:
            pass


In [ ]:
# Adiciona os tifs da pasta para o mapa

import os
import arcpy

def add_tifs_to_map(output_folder, mapa):
    """
    Percorre a pasta output_folder, lista todos os arquivos TIFF e os adiciona ao mapa especificado.
    
    Parâmetros:
      output_folder (str): Caminho para a pasta onde estão os arquivos TIFF.
      mapa: Objeto do mapa (arcgis.mp.ArcGISMap) ao qual os TIFF serão adicionados.
    """
    # Lista os arquivos na pasta output_folder que terminam com ".tif"
    tif_files = [f for f in os.listdir(output_folder) if f.lower().endswith(".tif")]
    if not tif_files:
        print("Nenhum arquivo TIFF foi encontrado na pasta de saída.")
        return

    print(f"{len(tif_files)} arquivo(s) TIFF encontrados. Adicionando ao mapa...")
    for tif in tif_files:
        full_path = os.path.join(output_folder, tif)
        try:
            # Adiciona o arquivo TIFF ao mapa
            layer = mapa.addDataFromPath(full_path)
            print(f"Arquivo {tif} adicionado com sucesso.")
        except Exception as e:
            print(f"Erro ao adicionar {tif}: {e}")

# Exemplo de uso:
# Supondo que você já tenha um objeto 'aprx' e que 'aprx.activeMap' represente o mapa ativo
output_folder = r"C:\Users\davils\Documents\ArcGIS\Projects\API\tempp"
mapa = aprx.activeMap  # ou outro mapa que você deseje usar
add_tifs_to_map(output_folder, mapa)
